<a href="https://colab.research.google.com/github/saivenkat2309/ML_A1/blob/main/ML_AS_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00
time: 453 µs (started: 2023-12-02 00:43:03 +00:00)


In [3]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

time: 12.6 s (started: 2023-12-02 00:43:14 +00:00)


In [4]:
torch.manual_seed(123)


time: 12.2 ms (started: 2023-12-02 00:43:34 +00:00)


In [5]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

100%|██████████| 170498071/170498071 [00:11<00:00, 14706947.66it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
time: 33.3 s (started: 2023-12-02 00:43:48 +00:00)


In [6]:
mean

tensor([0.4914, 0.4822, 0.4465])

time: 36.1 ms (started: 2023-12-02 00:44:28 +00:00)


In [7]:
std

tensor([0.2470, 0.2435, 0.2616])

time: 6.47 ms (started: 2023-12-02 00:44:30 +00:00)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time: 3.77 ms (started: 2023-12-02 00:44:41 +00:00)


In [9]:
device

device(type='cpu')

time: 3.18 ms (started: 2023-12-02 00:44:59 +00:00)


In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
])

time: 852 µs (started: 2023-12-02 00:45:01 +00:00)


In [11]:
cifar10 = datasets.CIFAR10(
    './data', train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


time: 649 ms (started: 2023-12-02 00:45:12 +00:00)


In [12]:
cifar10_val = datasets.CIFAR10(
     './data', train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

time: 504 ms (started: 2023-12-02 00:45:33 +00:00)


In [13]:
first_image, label = cifar10[0]
print(first_image.shape)

torch.Size([3, 32, 32])
time: 10.9 ms (started: 2023-12-02 00:45:44 +00:00)


In [14]:
train_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)

time: 1e+03 µs (started: 2023-12-02 00:45:56 +00:00)


In [15]:

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 10)
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

time: 18.8 ms (started: 2023-12-02 00:46:07 +00:00)


In [16]:
def train_model(model, train_loader, test_loader, num_epochs=300, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Testing the model
        model.eval()
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            accuracy = correct / total

        if epoch % 1 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Test Accuracy: {accuracy * 100:.2f}%')

    # Classification Report
    report = classification_report(all_labels, all_predicted)
    print("Classification Report:\n", report)

time: 7.22 ms (started: 2023-12-02 00:47:06 +00:00)


In [17]:
train_model(model, train_loader, test_loader, num_epochs=300, lr=0.01)

Epoch 1/300, Loss: 1.7886348081870638, Test Accuracy: 40.94%
Epoch 2/300, Loss: 1.6521172705020037, Test Accuracy: 43.15%
Epoch 3/300, Loss: 1.5804378465437692, Test Accuracy: 44.45%
Epoch 4/300, Loss: 1.5198366514246813, Test Accuracy: 45.26%
Epoch 5/300, Loss: 1.4625313013544161, Test Accuracy: 46.44%
Epoch 6/300, Loss: 1.4085316710646, Test Accuracy: 47.50%
Epoch 7/300, Loss: 1.3539841107580803, Test Accuracy: 47.11%
Epoch 8/300, Loss: 1.3031534532744078, Test Accuracy: 47.23%
Epoch 9/300, Loss: 1.249340126015632, Test Accuracy: 48.36%
Epoch 10/300, Loss: 1.1990976107097633, Test Accuracy: 47.90%
Epoch 11/300, Loss: 1.147438968516891, Test Accuracy: 48.39%
Epoch 12/300, Loss: 1.0958708692496966, Test Accuracy: 49.10%
Epoch 13/300, Loss: 1.0460041413990564, Test Accuracy: 48.73%
Epoch 14/300, Loss: 0.9943198018247931, Test Accuracy: 48.12%
Epoch 15/300, Loss: 0.9466876947223873, Test Accuracy: 48.06%
Epoch 16/300, Loss: 0.8974496146195681, Test Accuracy: 48.03%
Epoch 17/300, Loss: 0.

In [18]:
model2 = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 256),
    nn.Tanh(),
    nn.Linear(256, 128),
    nn.Tanh(),
    nn.Linear(128, 10)
).to(device)



time: 23.8 ms (started: 2023-12-02 03:54:42 +00:00)


In [ ]:
train_model(model2, train_loader, test_loader, num_epochs=300, lr=0.02)

Epoch 1/200, Loss: 1.8525115716785325, Test Accuracy: 39.86%
Epoch 2/200, Loss: 1.6869112958682324, Test Accuracy: 41.70%
Epoch 3/200, Loss: 1.6147569851927168, Test Accuracy: 43.42%
Epoch 4/200, Loss: 1.5524838497374809, Test Accuracy: 45.18%
Epoch 5/200, Loss: 1.4961205005111866, Test Accuracy: 45.93%
Epoch 6/200, Loss: 1.4420395591132396, Test Accuracy: 46.48%
Epoch 7/200, Loss: 1.3869562117395993, Test Accuracy: 47.04%
Epoch 8/200, Loss: 1.3304111210871261, Test Accuracy: 47.79%
Epoch 9/200, Loss: 1.2721629204539557, Test Accuracy: 47.46%
Epoch 10/200, Loss: 1.2145694345903184, Test Accuracy: 48.17%
Epoch 11/200, Loss: 1.1551353657238008, Test Accuracy: 46.41%
Epoch 12/200, Loss: 1.0963393020767167, Test Accuracy: 46.19%
Epoch 13/200, Loss: 1.0335245162382083, Test Accuracy: 47.70%
Epoch 14/200, Loss: 0.9775107119301536, Test Accuracy: 48.41%
Epoch 15/200, Loss: 0.915340742657601, Test Accuracy: 47.51%
Epoch 16/200, Loss: 0.8570451142310486, Test Accuracy: 47.54%
Epoch 17/200, Loss

In [ ]:
total_params = sum(p.numel() for p in model2.parameters() if p.requires_grad)
print("Total number of trainable parameters ", total_params,  '\n')